# Convolutional Neural Network on pixel neighborhoods

This notebook reads the pixel-neighborhood data written out by the Dataflow program of [1_explore.ipynb](./1_explore.ipynb) and trains a simple convnet model on Cloud ML Engine.


In [51]:
BUCKET = 'cloud-training-demos-ml'
PROJECT = 'cloud-training-demos'
REGION = 'us-central1'

In [52]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [53]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


In [ ]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

In [ ]:
%%bash
pip install --upgrade tensorflow

In [54]:
import tensorflow as tf
print(tf.__version__)

1.11.0


## Train CNN model locally

In [ ]:
%%bash
OUTDIR=${PWD}/cnn_trained
DATADIR=${PWD}/preproc/tfrecord
rm -rf $OUTDIR
gcloud ml-engine local train \
    --module-name=trainer.train_cnn --package-path=${PWD}/ltgpred/trainer \
    -- \
    --train_steps=10 --num_eval_records=512 --train_batch_size=16 --num_cores=1 --nlayers=5 \
    --job-dir=$OUTDIR --train_data_path=${DATADIR}/train* --eval_data_path=${DATADIR}/eval*

## Training lighting prediction model on CMLE using GPU

custom_model_m_gpu is a machine with 4 K-80 GPUs.

In [25]:
%writefile largemachine.yaml
trainingInput:
  scaleTier: CUSTOM
  masterType: complex_model_m_p100

Overwriting largemachine.yaml


In [ ]:
%%bash
OUTDIR=gs://${BUCKET}/lightning/cnn_trained_gpu
DATADIR=gs://$BUCKET/lightning/preproc/tfrecord
JOBNAME=ltgpred_cnn_$(date -u +%y%m%d_%H%M%S)
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
    --module-name=trainer.train_cnn --package-path=${PWD}/ltgpred/trainer --job-dir=$OUTDIR \
    --region=${REGION} --scale-tier=CUSTOM --config=largemachine.yaml \
    --python-version=3.5 --runtime-version=1.10 \
    -- \
    --train_data_path=${DATADIR}/train-* --eval_data_path=${DATADIR}/eval-* \
    --train_steps=5000 --train_batch_size=256 --num_cores=4 \
    --num_eval_records=128000 --nlayers=5 --dprob=0.05 --ksize=3 --nfil=10 --learning_rate=0.01

The training completed after 12 minutes with this result:
<pre>
loss: 0.3428 - acc: 0.8547 - mean_squared_error: 0.1059 - rmse: 0.2118 - val_loss: 0.3466 - val_acc: 0.8547 - val_mean_squared_error: 0.1068 - val_rmse: 0.2264
</pre>

## Training lightning prediction model on CMLE using TPUs

Next, let's generate more (8x) data and then train on the TPU.

In [ ]:
%%bash
OUTDIR=gs://${BUCKET}/lightning/cnn_trained_tpu
DATADIR=gs://$BUCKET/lightning/preproc/tfrecord
JOBNAME=ltgpred_cnn_$(date -u +%y%m%d_%H%M%S)
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
    --module-name=trainer.train_cnn --package-path=${PWD}/ltgpred/trainer --job-dir=$OUTDIR \
    --region=${REGION} --scale-tier=BASIC_TPU \
    --python-version=3.5 --runtime-version=1.9 \
    -- \
    --train_data_path=${DATADIR}/train* --eval_data_path=${DATADIR}/eval* \
    --train_steps=10000 --train_batch_size=1024 --num_cores=8  --use_tpu \
    --num_eval_records=128000 --nlayers=5 --dprob=0.05 --ksize=3 --nfil=10 --learning_rate=0.01

When I ran it, training finished with accuracy=

Copyright 2018 Google Inc. Licensed under the Apache License, Version 2.0 (the \"License\"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an \"AS IS\" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License